In [1]:
import numpy as np
import pandas as pd
import orca
# os.chdir('../')
# import os; os.chdir('/home/amelia/activitysynth/activitysynth/')
import os; os.chdir('/home/amelia/ual_model_workspace/spring-2019-models')

import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna

from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import time
# from scripts import datasources, models, variables, utils
# scripts = '~/ual_model_workspace/spring-2019-models/scripts'

In [2]:
print(os.getcwd())
import sys
print(sys.executable)

/home/amelia/ual_model_workspace/spring-2019-models
/home/amelia/enter/envs/template-env/bin/python


In [3]:
import scripts

In [4]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [5]:
scripts.__path__

['/home/amelia/ual_model_workspace/spring-2019-models/scripts']

In [6]:
import urbansim_templates
urbansim_templates.__version__


'0.1.1'

# Loading data

In [7]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 97.49 s
Total time to execute iteration 1 with iteration value None: 97.49 s


In [8]:
#Loading Data 
beam_skims = orca.get_table('beam_skims').to_frame()
reset_beam_skims = beam_skims.reset_index()

students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv').rename({"parcel_id": "school_parcel_id"}, axis=1)
parcels = orca.get_table('parcels').to_frame().reset_index()

#Preprocessing
students = students[students.AGE <= 18]

persons = orca.get_table("persons").to_frame().reset_index()
hh = orca.get_table("households").to_frame().reset_index()


In [48]:
#Adding list of grades offered by each school
list_grades = []
for index, row in schools.loc[:,schools.columns.str.startswith("grade_")].iterrows():
    x = np.array(row)
    list_grades.append(x)
    
schools['list_grades'] = list_grades

In [49]:
schools.head()

,CDSCode,School,District,County,Street,City,Zip,State,Latitude,Longitude,...,sw_rank_2108,sw_rank_2017,sw_rank_2016,ss_rank_2018_,ss_rank_2017,ss_rank_2016,nodeID,school_parcel_id,school_id,list_grades
0,1100170112607,Envision Academy for Arts & Technology,Alameda County Office of Education,Alameda,1515 Webster Street,Oakland,94612-3355,CA,37.804520,-122.26815,...,4.0,3.0,2.0,NaN,5.0,4.0,53082889,6103,1000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]"
1,1100170123968,Community School for Creative Education,Alameda County Office of Education,Alameda,2111 International Boulevard,Oakland,94606-4903,CA,37.784648,-122.23863,...,2.0,1.0,2.0,NaN,1.0,5.0,53087346,24358,1001,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]"
2,1100170124172,Yu Ming Charter,Alameda County Office of Education,Alameda,1086 Alcatraz Avenue,Oakland,94608-1265,CA,37.847375,-122.28356,...,10.0,10.0,10.0,NaN,6.0,6.0,53126420,23218,1002,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]"
3,1100170125567,Urban Montessori Charter,Alameda County Office of Education,Alameda,5328 Brann Street,Oakland,94619-3312,CA,37.778352,-122.18950,...,6.0,6.0,6.0,NaN,1.0,1.0,53097173,54245,1003,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]"
4,1100170130401,Alameda County Juvenile Hall/Court,Alameda County Office of Education,Alameda,2500 Fairmont Avenue,San Leandro,94578-1005,CA,37.712878,-122.11173,...,NaN,NaN,NaN,NaN,NaN,NaN,718372425,196545,1004,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]"


## Defining set choice contrains

In [10]:
#Small netwrok 
nodessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_nodes.csv').set_index('osmid')
edgessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_edges.csv')
# netsmall = pdna.Network(nodessmall.x, nodessmall.y, edgessmall.u,
#                                 edgessmall.v, edgessmall[['length']],
#                                 twoway=False)
# netsmall.precompute(25000)



In [11]:
def node_id_small(x, y, netsmall):
    """ Return the node ID given a pair of coordinates"""
    idssmall = netsmall.get_node_ids(x, y)
    return idssmall


In [12]:
#Merging students and schools 
students_1 = students.merge(schools, how = 'left', on = 'school_id').drop(["parcel_id_work", "zone_id_work", "nodeID"], axis=1)
# Droping students with no assigned school
students_1 = students_1.dropna(subset=['school_id'])

In [13]:
#Define the node ID for each home and school location in students dataset
students_1['node_id_home'] = node_id_small(students_1.HXCORD, students_1.HYCORD, netsmall)
students_1['node_id_school'] = node_id_small(students_1.Longitude, students_1.Latitude, netsmall)

#Creating a df for public shcools only
df_public = students_1[students_1.school_id <= 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

# #Creating a df for private shcools only
df_private = students_1[students_1.school_id > 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

In [14]:
#Setting public and private schools as POIs. 
netsmall.set_pois('public_school', 500000, 10000, schools[schools.type == 'public']["Longitude"], 
                  schools[schools.type == 'public']["Latitude"])


netsmall.set_pois('private_school', 500000, 10000, 
                  schools[schools.type == 'private'].Longitude, 
                  schools[schools.type == 'private'].Latitude)

In [15]:
#Public schools. 95% of the time, the school is within the x's closest schools. Finding x
n =50

# n closest public schools per each node in netsmall
distance_matrix_public = netsmall.nearest_pois(200000, 
                                               'public_school', 
                                               num_pois=n,
                                               include_poi_ids=True)

# Selects POIS id's only 
public_nodes = distance_matrix_public.iloc[:,n:]

# Creates a list of n closest POIs for each node
list_values = []
for index, row in public_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)

#Add created list to public nodes 
public_nodes['list_values'] = list_values


merge = df_public.merge(public_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.9131504922644164

In [16]:
#Private schools. 95% of the time, the school is within the x's closest schools. Finding x
n =100

distance_matrix_private = netsmall.nearest_pois(100000, 
                                                'private_school', 
                                                num_pois=n, 
                                                include_poi_ids=True)

private_nodes = distance_matrix_private.iloc[:,n:]

list_values = []
for index, row in private_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)
    
private_nodes['list_values'] = list_values


merge = df_private.merge(private_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.8758465011286681

In [17]:
students = students[students.AGE <= 18]

In [18]:
def school_available(age, grades_offered):
    """ Checks if a school offers a grade according to the age range
    Input: Age: int in range (5-18)
           grades_offered: 13 element array
    Output: True if grade if offered acoording to age, false otherwise. 
     """
    index = age - 6
    
    if index < 0:
        index = 0

    if index < 12:
        result = (grades_offered[index] == 1) | (grades_offered[index + 1] == 1)
        
    elif index == 12:
        result = (grades_offered[index] == 1)
    
    return result

In [19]:
def school_choice_set(house_node_id, kid_age):
    """ Determines the school choice set given home node id and age of the student (4-18)
    Output: Pandas series with available school IDs"""
    
    public_id = distance_matrix_public.iloc[:,50:].loc[house_node_id] + 1000
    private_id = distance_matrix_private.iloc[:,100:].loc[house_node_id] + 1000

    schools_filter = pd.concat([public_id, private_id])
    schools_set = schools[schools.school_id.isin(schools_filter)]
#     print(schools_set.columns)
#     print(type(schools_set))
#     print(schools_set)
    school_availability = [school_available(kid_age, x) for x in schools_set.list_grades]
    try:
        schools_available = schools_set[school_availability].school_id
        return schools_available
    except Exception as e:
        print(e)
        print("empty dataframe?")
    
    return np.nan

In [38]:
# school_choice_set(53082889, 13)

In [111]:
schools.columns

Index(['CDSCode', 'School', 'District', 'County', 'Street', 'City', 'Zip',
       'State', 'Latitude', 'Longitude', 'type', 'grade_0', 'grade_1',
       'grade_2', 'grade_3', 'grade_4', 'grade_5', 'grade_6', 'grade_7',
       'grade_8', 'grade_9', 'grade_10', 'grade_11', 'grade_12',
       'Kindergarten Enrollment', 'Grade 1 Enrollment', 'Grade 2 Enrollment',
       'Grade 3 Enrollment', 'Grade 4 Enrollment', 'Grade 5 Enrollment',
       'Grade 6 Enrollment', 'Grade 7 Enrollment', 'Grade 8 Enrollment',
       'Ungraded Elementary Enrollmnet', 'Grade 9 Enrollment',
       'Grade 10 Enrollment', 'Grade 11 Enrollment', 'Grade 12 Enrollment',
       'Ungraded Secondary Enrollment', 'Total Enrollment', 'sw_rank_2108',
       'sw_rank_2017', 'sw_rank_2016', 'ss_rank_2018_', 'ss_rank_2017',
       'ss_rank_2016', 'nodeID', 'school_parcel_id', 'school_id',
       'list_grades', 'foo'],
      dtype='object')

In [151]:
def get_parcel_id(school_id):
    try: 
#         parcel_id = (schools.iloc[school_id])["school_parcel_id"]
        parcel_id = schools[schools["school_id"] == school_id]["school_parcel_id"].values[0]
        
    except Exception as e:
        print(e)
        parcel_id = np.nan
#     print(parcel_id)
    return parcel_id

In [120]:
schools.head()

,CDSCode,School,District,County,Street,City,Zip,State,Latitude,Longitude,...,sw_rank_2017,sw_rank_2016,ss_rank_2018_,ss_rank_2017,ss_rank_2016,nodeID,school_parcel_id,school_id,list_grades,foo
0,1100170112607,Envision Academy for Arts & Technology,Alameda County Office of Education,Alameda,1515 Webster Street,Oakland,94612-3355,CA,37.804520,-122.26815,...,3.0,2.0,NaN,5.0,4.0,53082889,6103,1000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]",6103.0
1,1100170123968,Community School for Creative Education,Alameda County Office of Education,Alameda,2111 International Boulevard,Oakland,94606-4903,CA,37.784648,-122.23863,...,1.0,2.0,NaN,1.0,5.0,53087346,24358,1001,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]",NaN
2,1100170124172,Yu Ming Charter,Alameda County Office of Education,Alameda,1086 Alcatraz Avenue,Oakland,94608-1265,CA,37.847375,-122.28356,...,10.0,10.0,NaN,6.0,6.0,53126420,23218,1002,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]",NaN
3,1100170125567,Urban Montessori Charter,Alameda County Office of Education,Alameda,5328 Brann Street,Oakland,94619-3312,CA,37.778352,-122.18950,...,6.0,6.0,NaN,1.0,1.0,53097173,54245,1003,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]",NaN
4,1100170130401,Alameda County Juvenile Hall/Court,Alameda County Office of Education,Alameda,2500 Fairmont Avenue,San Leandro,94578-1005,CA,37.712878,-122.11173,...,NaN,NaN,NaN,NaN,NaN,718372425,196545,1004,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]",NaN


In [106]:
def get_zone_id(parcel_id):
    '''gets the zone_id (TAZ) of school and home locations by the parcel id'''
    try: 
        zone_id = (parcels.iloc[parcel_id])["zone_id"]

    except Exception as e:
#         print(e)
        zone_id = np.nan
    return zone_id
 

In [98]:
# schools["school_parcel_id"].apply(get_zone_id)

### Create parameters table with distances and generalized cost

In [ ]:
stu = students_1[["node_id_home", "AGE", "school_parcel_id", "parcel_id_home"]].head(1000)
dfs =[]
for index, record in stu.iterrows():
    age = int(record["AGE"])
    parcel_id_home = int(record["parcel_id_home"])
    node_id_home = int(record["node_id_home"])
    school_choices = school_choice_set(node_id_home, age)
    school_parcel_id = [schools[schools["school_id"] == choice]["school_parcel_id"].values[0] for choice in school_choices]
    df = pd.DataFrame({"node_id_home": node_id_home, "parcel_id_home": parcel_id_home, "age": age,
                       "school_choice": school_choices, "parcel_id_school": school_parcel_id})
    df["school_zone_id"] = df["parcel_id_school"].apply(get_zone_id)
    df["home_zone_id"] = df["parcel_id_home"].apply(get_zone_id)

    
    dfs.append(df)
    
all_choices = pd.concat(dfs, axis=0)


In [ ]:
all_choices_costs = pd.merge(all_choices, reset_beam_skims,  how='left', left_on=['school_zone_id','home_zone_id'], right_on = ['to_zone_id','from_zone_id'])
all_choices_costs = all_choices_costs.set_index("node_id_home")
all_choices_costs.head()
all_choices_costs.to_csv("all_choices_costs.csv")


In [ ]:
print(len(all_choices))
print(len(all_choices_costs))

In [ ]:
all_choices_costs.head()

### Prepare test data from PUMS synthetic population

In [33]:
per_hh = pd.merge(hh, persons, on='household_id')
pums_students = per_hh[(per_hh["age"] >= 5) & (per_hh["age"] <= 18) & (per_hh["student"] ==1)]


stu_pums = pums_students[["household_id", 'node_id_small_x', "age", "zone_id_home", 'hh_inc_under_25k', 'hh_inc_25_to_75k', 'hh_inc_75_to_200k']] 
test = stu_pums.sample(1000)
test2 = stu_pums.sample(200)
test3 = stu_pums.sample(10000)
test4 = stu_pums.sample(100000)


In [23]:
persons.head()

,member_id,Unnamed: 0,student,household_id,age,primary_commute_mode,worker,work_at_home,sex,hours,...,zone_id_home,lessGED,GED,somebach,Assoc,Bach,female,white,minority,age_16less25
0,1,0,0,0,47,1.0,1,0,1,40.0,...,557.0,0,0,0,0,0,0,1,0,0
1,1,1,0,670,47,1.0,1,0,1,40.0,...,539.0,0,0,0,0,0,0,1,0,0
2,1,2,0,740,47,1.0,1,0,1,40.0,...,539.0,0,0,0,0,0,0,1,0,0
3,1,3,0,975,47,1.0,1,0,1,40.0,...,558.0,0,0,0,0,0,0,1,0,0
4,1,4,0,977,47,1.0,1,0,1,40.0,...,558.0,0,0,0,0,0,0,1,0,0


In [30]:
# school_choices = test.apply(lambda x: school_choice_set(x["node_id_small_x"], x["age"]), axis=1)
# school_choices


In [152]:
def table_cols(record):
    age = int(record["age"])
    node_id_home = int(record["node_id_small_x"])
    zone_id_home = int(record["zone_id_home"])
    hh_id = int(record["household_id"])
    hh_inc_under_25k = record["hh_inc_under_25k"]
    hh_inc_25_to_75k = record["hh_inc_25_to_75k"]
    hh_inc_75_to_200k = record["hh_inc_75_to_200k"]
    
    school_choices = school_choice_set(node_id_home, age)
    
    d = {"node_id_home": node_id_home, "zone_id_home": zone_id_home, 
                        "age": age, "household_id": hh_id,
                       "school_choice_option": school_choices, "hh_inc_under_25k": hh_inc_under_25k, 
                       "hh_inc_25_to_75k": hh_inc_25_to_75k, "hh_inc_75_to_200k": hh_inc_75_to_200k}
    df = pd.DataFrame(d)
    df.columns = ["node_id_home", "zone_id_home", "age", "household_id",
                  "school_choice_option", "hh_inc_under_25k",  "hh_inc_25_to_75k",  "hh_inc_75_to_200k"]
    return df


In [159]:
def construct(df):
    print(".apply ")
    df1 = df.apply(table_cols, axis=1)
    
    print("concatenating")
    df2 = pd.concat(df1.values)
    print("vector opertions parcel and zone")
    df2["school_parcel_id"] = df2["school_choice_option"].apply(get_parcel_id)
    df2["school_zone_id"] = df2["school_parcel_id"].apply(get_zone_id)
    
    print("merge with beam skims")
    df3 = pd.merge(df2, beam_skims, left_on=['school_zone_id','zone_id_home'], right_on =['to_zone_id','from_zone_id'])
        
    return df3
 
    

In [160]:
start = time.time()
table = construct(test2)
print("writing to .csv")
table.to_csv("/home/amelia/activitysynth/activitysynth/output/SLCM_validation_test.csv")
print("wrote to .csv")


print("sample size: ", len(test2))
# print("table columns: ", table.columns)
print("table len: ", len(table))
end = time.time()
print("elapsed: ", end-start)

.apply 
concatenating
vector opertions parcel and zone
merge with beam skims
writing to .csv
wrote to .csv
sample size:  200
table len:  16401
elapsed:  23.49168872833252


In [158]:
table.head()

,node_id_home,zone_id_home,age,household_id,school_choice_option,hh_inc_under_25k,hh_inc_25_to_75k,hh_inc_75_to_200k,school_parcel_id,school_zone_id
414,1373640503,1210,10,1727629,1414,0.0,1.0,0.0,599236,1268.0
438,1373640503,1210,10,1727629,1438,0.0,1.0,0.0,615750,1186.0
486,1373640503,1210,10,1727629,1486,0.0,1.0,0.0,631511,892.0
493,1373640503,1210,10,1727629,1493,0.0,1.0,0.0,634863,1451.0
494,1373640503,1210,10,1727629,1494,0.0,1.0,0.0,595257,665.0


In [150]:
# blah = table["school_choice_option"].apply(lambda code: get_parcel_id(code))
# blah


In [87]:
ratio = 228.5/10000
print("len: ", len(stu_pums))
print("seconds: ", (len(stu_pums)*228)/10000)
print("mins: ", ((len(stu_pums)*228)/10000)/60)
print("hrs: ",(((len(stu_pums)*228)/10000)/60)/60)



len:  1319604
seconds:  30086.9712
mins:  501.44952
hrs:  8.357492


In [86]:
(len(stu_pums)*228)/10000

30086.9712

In [54]:
import time
start1 = time.time()
# stu_pums = pums_students[["household_id", 'node_id_small_x', "age", "zone_id_home", 'hh_inc_under_25k', 'hh_inc_25_to_75k', 'hh_inc_75_to_200k']] 
# sample = stu_pums.head(10000)
dfs =[]
for index, record in test.iterrows():
    age = int(record["age"])
#     parcel_id_home = int(record["parcel_id_home"])
    node_id_home = int(record["node_id_small_x"])
    zone_id_home = int(record["zone_id_home"])
    hh_id = int(record["household_id"])
    hh_inc_under_25k = record["hh_inc_under_25k"]
    hh_inc_25_to_75k = record["hh_inc_25_to_75k"]
    hh_inc_75_to_200k = record["hh_inc_75_to_200k"]
    
    school_choices = school_choice_set(node_id_home, age)
    school_parcel_id = [schools[schools["school_id"] == choice]["school_parcel_id"].values[0] for choice in school_choices]

    df = pd.DataFrame({"node_id_home": node_id_home, "zone_id_home": zone_id_home, 
                        "age": age, "household_id": hh_id, "school_parcel_id": school_parcel_id,
                       "school_choice_option": school_choices, "hh_inc_under_25k": hh_inc_under_25k, 
                       "hh_inc_25_to_75k": hh_inc_25_to_75k, "hh_inc_75_to_200k": hh_inc_75_to_200k})
    df["zone_id_school"] = df["school_parcel_id"].apply(get_zone_id)

    dfs.append(df)
#     "zone_id_school": zone_id_school,
all_choices_pums = pd.concat(dfs, axis=0)
all_choices_costs = pd.merge(all_choices_pums, reset_beam_skims,  how='left', left_on=['zone_id_school','zone_id_home'], right_on = ['to_zone_id','from_zone_id'])
# all_choices_costs = all_choices_costs.set_index("node_id_home")
end1 = time.time()
print("elapsed: ", end1-start1)
all_choices_costs.head()



elapsed:  108.88053297996521


,node_id_home,zone_id_home,age,household_id,school_parcel_id,school_choice_option,hh_inc_under_25k,hh_inc_25_to_75k,hh_inc_75_to_200k,zone_id_school,...,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
0,65457292,227,8,1458712,1131179,1968,0.0,1.0,1.0,708.0,...,19328.030019,7475.696414,545.904976,117.280484,194.216436,210.287486,163.395508,185.464429,2302.143885,7808.592532
1,65457292,227,8,1458712,1060918,1978,0.0,1.0,1.0,221.0,...,2036.155099,787.544171,57.509596,12.355178,20.460170,22.153212,17.213270,19.538170,242.524562,822.613866
2,65457292,227,8,1458712,1066783,1980,0.0,1.0,1.0,1443.0,...,6902.019325,2669.563378,194.942097,41.880738,69.354486,75.093442,58.348365,66.229154,822.093176,2788.440234
3,65457292,227,8,1458712,1046493,1983,0.0,1.0,1.0,1079.0,...,9572.542464,3702.468452,270.368918,58.085196,96.189062,104.148529,80.924462,91.854479,1140.176732,3867.341034
4,65457292,227,8,1458712,1059140,1984,0.0,1.0,1.0,179.0,...,3715.923657,1437.245137,104.953335,22.547840,37.339214,40.428965,31.413715,35.656591,442.600250,1501.246309


In [79]:
# all_choices_costs = pd.merge(all_choices_pums, reset_beam_skims,  how='left', left_on=['zone_id_school','zone_id_home'], right_on = ['to_zone_id','from_zone_id'])
# # all_choices_costs = all_choices_costs.set_index("node_id_home")
# end = time.time()
# print("elapsed: ", end-start)
# all_choices_costs.to_csv("all_choices_costs.csv")



elapsed:  344.1161913871765


In [67]:
len(stu_pums.)

1319604

In [66]:
table.head()

,node_id_home,zone_id_home,age,household_id,school_choice_option,hh_inc_under_25k,hh_inc_25_to_75k,hh_inc_75_to_200k
968,65457292,227,8,1458712,1968,0.0,1.0,1.0
978,65457292,227,8,1458712,1978,0.0,1.0,1.0
980,65457292,227,8,1458712,1980,0.0,1.0,1.0
983,65457292,227,8,1458712,1983,0.0,1.0,1.0
984,65457292,227,8,1458712,1984,0.0,1.0,1.0


In [69]:
##try with less data

In [73]:
test.iloc[0].index

Index(['household_id', 'node_id_small_x', 'age', 'zone_id_home',
       'hh_inc_under_25k', 'hh_inc_25_to_75k', 'hh_inc_75_to_200k'],
      dtype='object')

In [ ]:
def table_cols(record):
    person_id = record.index
    age = int(record["age"])
#     parcel_id_home = int(record["parcel_id_home"])
    node_id_home = int(record["node_id_small_x"])
#     zone_id_home = int(record["zone_id_home"])
#     hh_id = int(record["household_id"])
#     hh_inc_under_25k = record["hh_inc_under_25k"]
#     hh_inc_25_to_75k = record["hh_inc_25_to_75k"]
#     hh_inc_75_to_200k = record["hh_inc_75_to_200k"]
    
    # print("get choices")
    school_choices = school_choice_set(node_id_home, age)
    
    # print(len(school_choices), " number of choices")
#     school_parcel_ids = [schools[schools["school_id"] == choice]["school_parcel_id"].values[0] for choice in school_choices]
#     school_zone_ids = [get_zone_id(sid) for sid in school_parcel_ids]
#     # df["zone_id_school"] = df["school_parcel_id"].apply(get_zone_id)
#     return pd.Series([age, hh_id, node_id_home, zone_id_home, school_parcel_id, school_zone_id, hh_inc_under_25k, hh_inc_25_to_75k, hh_inc_75_to_200k])
    d = {"node_id_home": node_id_home, "zone_id_home": zone_id_home, 
                        "age": age, "household_id": hh_id,
                       "school_choice_option": school_choices, "hh_inc_under_25k": hh_inc_under_25k, 
                       "hh_inc_25_to_75k": hh_inc_25_to_75k, "hh_inc_75_to_200k": hh_inc_75_to_200k}
    df = pd.DataFrame(d)
    df.columns = ["node_id_home", "zone_id_home", "age", "household_id",
                  "school_choice_option", "hh_inc_under_25k",  "hh_inc_25_to_75k",  "hh_inc_75_to_200k"]
#     df["zone_id_school"] = df["school_parcel_id"].apply(get_zone_id)
    return df


# Additional code

In [ ]:
# for table_name in orca.list_tables():
#     print(table_name.upper())
#     print(orca.get_table(table_name).to_frame().columns.tolist())
#     print()

In [ ]:
# #Creating school availability matrix based on distance
# school_availability_distance = pd.DataFrame(nodessmall.index)
# for x in range(1000,3547,1):
#     name = "school_" + str(x)
#     school_availability_distance[name] = x
# school_availability_distance = school_availability_distance.set_index('osmid')

In [ ]:
# school_availability_distance

In [ ]:

# a = students.sample(10)
# a.loc[:,['AGE','HCITY','HYCORD','HXCORD', 'school_id']]

In [ ]:
# students[(students.SAMPN == 7176467) & (students.PERNO ==4) ][['HCITY', 'HYCORD','HXCORD',
#                                                                 'AGE','SNAME_lookup','SCITY_lookup', 'school_id']]

In [ ]:
# schools[schools.school_id == 1271]

In [ ]:
# d[d.type == 'public'].sort_values('dist', ascending=False)

In [ ]:
# os.listdir("/home/data/fall_2018/")

In [ ]:
# # Estimating 95th percentail of distances between home and school
# a = schools.merge(parcels[['zone_id']], left_on ='parcel_id', right_index=True)
# b = students[['SAMPN','PERNO','zone_id_home', 'school_id']].merge(a[['zone_id','school_id','type']], on = 'school_id')
# c = beam_skims.reset_index(level=['from_zone_id', 'to_zone_id'])
# d = b.merge(c.loc[:,['from_zone_id','to_zone_id', 'dist']], 
#             left_on = ['zone_id_home','zone_id'], 
#             right_on = ['from_zone_id', 'to_zone_id'])

# public_dist_limit = d[d.type == 'public'].quantile(1).dist
# private_dist_limit = d[d.type == 'private'].quantile(1).dist

# sns.distplot(d[d.type == 'public'].dist,kde=False)
# sns.distplot(d[d.type == 'private'].dist,kde=False);
# d.sort_values('dist', ascending=False)

In [ ]:
# a = students.loc[:,['HCITY','SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup',
#                 'SXCORD_lookup', 'SYCORD_lookup', 'SPrimaryCity_lookup',]]
# same_city = []
# for index, row in a.iterrows():
#     if row['HCITY'].lower() == row['SCITY_lookup']:
#         same_city.append(True)
#     else:
#         same_city.append(False)

# a['same_city'] = same_city
# a.same_city.mean()

In [ ]:
#'network_aggregations_small'])
# orca.run(['initialize_network_walk', 'network_aggregations_walk'])

# orca.get_table('nodeswalk').to_frame().to_csv('./data/walk_net_vars.csv')
# orca.get_table('nodessmall').to_frame().to_csv('./data/drive_net_vars.csv')
# orca.get_table('zones').to_frame().to_csv('./data/zones_w_access_vars-2025-b-lt.csv')

In [ ]:
# os.listdir("/home/data/spring_2019/2025")

In [ ]:
# a = students.loc[:,['SAMPN', 'PERNO', 'AGE', 'HPrimaryCity','school_id']].merge(schools.loc[:,['school_id','City','District']], how = 'inner', on = 'school_id')
# students[students.SAMPN == 1342402].loc[:,['SNAME_lookup','school_id', 'HPrimaryCity']]
# schools[schools.school_id == 1710]